In [1]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import time
import tkinter as tk
from tkinter import messagebox, filedialog
from threading import Thread

Implementation of Lucas-Kanade Optical Flow with Basic Movements 

In [17]:
ix, iy, k = 200, 200, 1

def onMouse(event, x, y, flags, param):
    global ix, iy, k
    if event == cv2.EVENT_LBUTTONDOWN:
        ix, iy = x, y
        k = -1

cv2.namedWindow("window")
cv2.setMouseCallback("window", onMouse)

cap = cv2.VideoCapture(0)

while True:
    _, frame = cap.read()
    cv2.imshow("window", frame)

    if cv2.waitKey(1) == 27 or k == -1:  # 27 is code for esc key
        old_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        break

old_points = np.array([[ix, iy]], dtype="float32").reshape(-1, 1, 2)
mask = np.zeros_like(frame)

while True:
    _, frame2 = cap.read()

    if frame2 is None:  # Check if frame2 is valid
        break

    new_frame = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

    new_points, status, error = cv2.calcOpticalFlowPyrLK(old_frame, new_frame, old_points,
                                                         None, maxLevel=1,
                                                         criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT,
                                                                   15, 0.08))
    
    cv2.circle(mask, (int(new_points.ravel()[0]), int(new_points.ravel()[1])), 2, (0, 255, 0), 2)
    combined = cv2.addWeighted(frame2, 0.7, mask, 0.3, 0.1)

    cv2.imshow("new window", mask)
    cv2.imshow("window", combined)

    old_frame = new_frame.copy()
    old_points = new_points.copy()

    if cv2.waitKey(1) == 27:  # 27 is code for esc key
        break

# Release video capture and close windows
cap.release()
cv2.destroyAllWindows()

Integration of Lucas-Kanade but tracking selected points only (no trailing lines)

In [20]:
#accuracy is a bit low

ix, iy, k = 200, 200, 1

def onMouse(event, x, y, flags, param):
    global ix, iy, k
    if event == cv2.EVENT_LBUTTONDOWN:
        ix, iy = x, y
        k = -1

cv2.namedWindow("window")
cv2.setMouseCallback("window", onMouse)

cap = cv2.VideoCapture(0)

while True:
    _, frame = cap.read()
    cv2.imshow("window", frame)

    if cv2.waitKey(1) == 27 or k == -1:  # 27 is code for esc key
        old_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        break

old_points = np.array([[ix, iy]], dtype="float32").reshape(-1, 1, 2)

while True:
    _, frame2 = cap.read()

    if frame2 is None:  # Check if frame2 is valid
        break

    new_frame = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

    new_points, status, error = cv2.calcOpticalFlowPyrLK(old_frame, new_frame, old_points,
                                                         None, maxLevel=1,
                                                         criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT,
                                                                   15, 0.08))

    # Draw a slightly bigger dot at the new point
    if status.ravel()[0] == 1:  # Check if the point is successfully tracked
        x, y = new_points.ravel()
        mask = np.zeros_like(frame2)
        cv2.circle(mask, (int(x), int(y)), 5, (0, 255, 0), -1)  # Increase the radius to make it bigger
        combined = cv2.addWeighted(frame2, 0.7, mask, 0.3, 0.1)
        cv2.imshow("window", combined)

    old_frame = new_frame.copy()
    old_points = new_points.copy()

    if cv2.waitKey(1) == 27:  # 27 is code for esc key
        break

# Release video capture and close windows
cap.release()
cv2.destroyAllWindows()

Improving tracking accuracy

In [15]:
ix, iy, k = 200, 200, 1

def onMouse(event, x, y, flags, param):
    global ix, iy, k
    if event == cv2.EVENT_LBUTTONDOWN:
        ix, iy = x, y
        k = -1

cv2.namedWindow("window")
cv2.setMouseCallback("window", onMouse)

cap = cv2.VideoCapture(0)

while True:
    _, frame = cap.read()
    cv2.imshow("window", frame)

    if cv2.waitKey(1) == 27 or k == -1:  # 27 is code for esc key
        old_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        break

patch_size = 21
half_patch = patch_size//2

old_points = np.array([[ix, iy]], dtype="float32").reshape(-1, 1, 2)
mask = np.zeros_like(frame)

while True:
    _, frame2 = cap.read()

    if frame2 is None:  # Check if frame2 is valid
        break

    new_frame = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

    #calculate the region of interest(ROI)
    x,y = old_points.ravel()
    x,y = int(x), int(y)
    roi = new_frame[y - half_patch:y + half_patch, x - half_patch:x + half_patch]

    new_x, new_y = cv2.minMaxLoc(roi)[3]
    new_x += x - half_patch
    new_y += y - half_patch

    new_points = np.array([[new_x, new_y]], dtype="float32").reshape(-1, 1, 2)

    # Draw a slightly bigger dot at the new point
    cv2.circle(frame2, (int(new_x), int(new_y)), 5, (0, 255, 0), -1)
    cv2.imshow("window", frame2)

    old_frame = new_frame.copy()
    old_points = new_points.copy()

    # new_points, status, error = cv2.calcOpticalFlowPyrLK(old_frame, new_frame, old_points,
    #                                                      None, maxLevel=3,
    #                                                      criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT,
    #                                                                30, 0.01))

    # # Draw a slightly bigger dot at the new point
    # if status.ravel()[0] == 1:  # Check if the point is successfully tracked
    #     x, y = new_points.ravel()
    #     mask = np.zeros_like(frame2)
    #     cv2.circle(mask, (int(x), int(y)), 5, (0, 255, 0), -1)  # Increase the radius to make it bigger
    #     combined = cv2.addWeighted(frame2, 0.7, mask, 0.3, 0.1)
    #     cv2.imshow("window", combined)

    # old_frame = new_frame.copy()

    if cv2.waitKey(1) == 27:  # 27 is code for esc key
        break

# Release video capture and close windows
cap.release()
cv2.destroyAllWindows()